In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
import os
from datetime import date
from os import listdir
from os.path import isfile, join
import math
from scipy.optimize import curve_fit


def d_day_index(f, n):
    for i1 in range(n):
        f.loc[i1, ['day_index']] = i1+1
    return f


def state_hour(f, n):
    for i2 in range(n + 1):
        dn = 24 * i2
        f.loc[dn:dn + 6, ['state']] = str('dark')
        f.loc[dn + 7:dn + 18, ['state']] = str('light')
        f.loc[dn + 19:dn + 24, ['state']] = str('dark')
    return f


def m_day_and_hour_index(f, n):
    h_counter = 1
    for i3 in range(n + 1):
        dn = 24 * i3
        f.loc[dn:dn + 23, ['day_index']] = i3+1
        for j3 in range(24):
            interval_m = dn + j3
            f.loc[interval_m:interval_m + 1, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f


def state_(f, n):
    for i4 in range(n + 1):
        dn = 1440 * i4
        f.loc[dn:dn + 419, ['state']] = str('dark')
        f.loc[dn + 420:dn + 1139, ['state']] = str('light')
        f.loc[dn + 1140:dn + 1439, ['state']] = str('dark')
    return f


def day_and_hour_index(f, n):
    h_counter = 1
    for i5 in range(n + 1):
        dn = 1440 * i5
        f.loc[dn:dn + 1439, ['day_index']] = i5+1
        for j5 in range(24):
            interval_m = dn + (j5*60)
            f.loc[interval_m:interval_m + 60, ['hour_index']] = h_counter
            h_counter = h_counter + 1
    return f

def threshup_():
    #DOI 10.1211/jpp.60.1.0008
    th_u_comft = 3.38 + 0.52
    #th_u_stretched = 4.63 + 0.44
    #th_u_burst = 6.63 + 0.92
    return th_u_comft


def real_consump(val, w, p):
    a = np.nan
    if (p == 5):
        a = (val*0.05*0.789)/(w) *1000
    elif (p == 10):
        a = (val*0.1*0.789)/(w) *1000
    elif ( p == 20 ):
        a = (val*0.2*0.789)/(w) *1000
    else:
        print("You messed up the input in real_consump function")
    return(a)



def wistar_w(g, h):
    # https://animal.ncku.edu.tw/p/412-1130-16363.php?Lang=en
    def fsigmoid(x, a, b):
        return  a + b * np.log(x)
    if g == str("male"):
        x = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        y = [48, 79, 116, 149, 195, 240, 291, 231, 342, 362]
        sigma = [5.6, 8.8, 12, 16, 20, 22, 22, 22, 22, 22]
    elif g == str("female"):
        x = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        y = [38, 65, 96, 122, 156, 182, 197, 218, 229, 240]
        sigma = [3, 4, 5, 8, 8, 7, 9, 7, 9, 17]
    else:
        print("You fucked up the gender on weight optimizer.")
    popt, pcov = curve_fit(fsigmoid, x, y)
    a, b = popt
    _x = math.ceil(((21*24)+h)/(24*7))
    #print(_x)
    return a + b * np.log(_x)

In [ ]:
def loco_writer(animal, dataframe, path):
    
    if animal == 1 :
        file_skipped = pd.read_excel("{}/loco/01/1 25.03 from 2000.Jan.03  00-52-32 to 2000.Feb.14  18-42-32 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/01/1 from 2020.Mrz.25  13-21-39 to 2020.Mai.17  05-46-39 -- 1.xlsx".format(path), skiprows=18)  
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/01/1 from 2020.Mai.19  14-19-16 to 2020.Jul.20  01-54-16 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/01/1 from 2020.Jul.22  13-10-11 to 2020.Sep.25  17-55-11 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/01/1 from 2020.Sep.30  14-52-56 to 2020.Nov.13  05-17-56 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
    elif (animal == 2):
        file_skipped = pd.read_excel("{}/loco/02/2 from 2020.Feb.17  13-24-00 to 2020.Mrz.03  01-54-00 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/02/2 from 2020.Aug.17  09-59-23 to 2020.Sep.24  07-54-23 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/02/2 from 2020.Sep.30  14-53-19 to 2020.Okt.02  11-13-19 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif (animal == 3):
        file_skipped = pd.read_excel("{}/loco/03/3 from 2020.Feb.05  14-42-39 to 2020.Mai.27  11-32-39 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/03/3 from 2020.Jun.04  14-22-29 to 2020.Sep.26  08-17-29 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/03/3 from 2020.Sep.30  14-53-47 to 2020.Nov.18  19-03-47 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
    elif (animal == 4):
        file_skipped = pd.read_excel("{}/loco/04/4 04.06 from 2000.Jan.03  00-50-54 to 2000.Apr.12  20-40-54 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/04/4 from 2020.Jun.04  14-22-51 to 2020.Sep.26  08-17-51 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/04/4 from 2020.Sep.30  14-54-05 to 2020.Nov.18  19-04-05 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 5):
        file_skipped = pd.read_excel("{}/loco/05/5 from 2020.Feb.05  14-42-42 to 2020.Apr.03  10-32-42 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/05/5 from 2020.Apr.15  15-54-52 to 2020.Jul.25  20-19-52 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/05/5 from 2020.Jul.30  09-42-19 to 2020.Nov.18  19-02-19 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 6):
        file_skipped = pd.read_excel("{}/loco/06/6 from 2020.Feb.05  14-42-39 to 2020.Mai.29  08-37-39 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/06/6 from 2020.Jun.04  14-23-14 to 2020.Sep.17  20-03-14 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/06/6 from 2020.Sep.23  11-56-34 to 2020.Nov.18  19-01-34 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 8):
        file_skipped = pd.read_excel("{}/loco/08/8 from 2020.Feb.05  14-42-50 to 2020.Apr.20  13-02-50 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/08/8 from 2020.Apr.21  14-03-12 to 2020.Aug.13  07-58-12 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/08/8 from 2020.Aug.17  09-59-46 to 2020.Nov.16  11-54-46 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 9):
        file_skipped = pd.read_excel("{}/loco/09/9 from 2020.Feb.05  14-42-37 to 2020.Mrz.09  22-22-37 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/09/9 from 2020.Mai.19  14-18-57 to 2020.Jun.26  11-03-57 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/09/9 from 2020.Jun.29  10-19-28 to 2020.Okt.10  16-04-28 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity


        file_skipped = pd.read_excel("{}/loco/09/9 from 2020.Mrz.18  10-48-09 to 2020.Mai.18  17-18-09 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/09/9 from 2020.Okt.14  11-11-25 to 2020.Nov.18  18-56-25 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 10):
        file_skipped = pd.read_excel("{}/loco/10/10 from 2020.Feb.05  14-42-45 to 2020.Mai.04  03-37-45 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/10/10 from 2020.Aug.11  17-00-25 to 2020.Aug.25  12-25-25 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/10/10 from 2020.Aug.31  11-35-38 to 2020.Nov.18  19-00-38 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/10/10 from 2020.Mai.06  14-57-14 to 2020.Aug.04  01-12-14 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity


    elif ( animal == 11):
        file_skipped = pd.read_excel("{}/loco/11/11 from 2020.Feb.05  14-43-04 to 2020.Mai.18  06-33-04 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/11/11 from 2020.Mai.19  14-18-39 to 2020.Jul.12  07-13-39 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/11/11 from 2020.Jul.22  13-10-30 to 2020.Nov.13  07-05-30 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif( animal == 12):
        file_skipped = pd.read_excel("{}/loco/12/12 from 2020.Feb.05  14-43-09 to 2020.Mai.05  17-33-09 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/12/12 from 2020.Jun.09  14-34-13 to 2020.Okt.01  08-29-13 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 13):
        file_skipped = pd.read_excel("{}/loco/13/13 from 2020.Feb.05  14-43-05 to 2020.Mai.05  23-18-05 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/13/13 from 2020.Mai.06  14-57-32 to 2020.Mai.26  15-42-32 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/13/13 from 2020.Jun.04  14-23-31 to 2020.Aug.01  11-28-31 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/13/13 from 2020.Aug.11  17-00-43 to 2020.Nov.18  19-00-43 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 14):
        file_skipped = pd.read_excel("{}/loco/14/14 from 2020.Feb.05  14-43-15 to 2020.Mai.06  03-38-15 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/14/14 Sw-7 - from 2020.Aug.12  10-08-25 to 2020.Nov.03  16-43-25 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/14/14 Sw-7 - from 2020.Nov.05  16-41-53 to 2020.Nov.18  17-56-53 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 15):
        file_skipped = pd.read_excel("{}/loco/15/15 25.03 from 2000.Jan.03  00-50-15 to 2000.Feb.19  15-40-15 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/15/15 from 2020.Mrz.25  13-21-06 to 2020.Jun.09  12-11-06 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/15/15 from 2020.Jun.09  14-34-34 to 2020.Jul.24  00-49-34 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/15/15 from 2020.Aug.04  09-54-00 to 2020.Nov.18  19-14-00 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 17):
        file_skipped = pd.read_excel("{}/loco/17/17 from 2020.Feb.05  14-43-28 to 2020.Mai.04  06-03-28 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/17/17 from 2020.Mai.06  14-58-14 to 2020.Aug.28  08-53-14 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/17/17 from 2020.Sep.02  11-50-53 to 2020.Nov.18  19-10-53 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 18):
        file_skipped = pd.read_excel("{}/loco/18/18 from 2020.Feb.05  14-43-17 to 2020.Mai.03  02-38-17 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/18/18 from 2020.Mai.06  14-58-34 to 2020.Mai.18  02-03-34 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/18/18 from 2020.Jun.01  11-18-03 to 2020.Sep.23  05-13-03 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/18/18 from 2020.Sep.30  14-54-25 to 2020.Nov.18  19-04-25 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 19):
        file_skipped = pd.read_excel("{}/loco/19/19 from 2020.Feb.17  13-22-51 to 2020.Mai.21  18-32-51 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/19/19 from 2020.Jun.04  14-23-52 to 2020.Sep.24  15-03-52 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/19/19 from 2020.Sep.30  14-54-41 to 2020.Nov.18  19-04-41 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 20):
        file_skipped = pd.read_excel("{}/loco/20/20 19.05 from 2000.Jan.03  00-48-52 to 2000.Apr.06  23-48-52 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/20/20 from 2020.Mai.19  14-19-46 to 2020.Sep.10  08-14-46 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/20/20 from 2020.Sep.30  14-54-59 to 2020.Nov.18  18-59-59 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 21):
        file_skipped = pd.read_excel("{}/loco/21/21 from 2020.Feb.05  14-43-18 to 2020.Mai.12  14-53-18 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/21/21 from 2020.Mai.19  14-20-11 to 2020.Sep.07  23-55-11 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/21/21 from 2020.Sep.14  13-47-21 to 2020.Nov.18  18-57-21 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 22):
        file_skipped = pd.read_excel("{}/loco/22/22 from 2020.Aug.17  10-03-58 to 2020.Nov.11  22-28-58 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 23):
        file_skipped = pd.read_excel("{}/loco/23/23 06.05 from 2000.Jan.03  00-45-00 to 2000.Mrz.22  08-45-00 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/23/23 from 2020.Mai.06  14-59-11 to 2020.Jul.14  15-24-11 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/23/23 from 2020.Aug.04  10-31-07 to 2020.Nov.14  23-11-07 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
    elif ( animal == 24):
        file_skipped = pd.read_excel("{}/loco/24/24 from 2020.Feb.10  12-05-42 to 2020.Jun.03  06-00-42 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/24/24 from 2020.Jun.04  14-24-11 to 2020.Sep.10  18-34-11 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/24/24 from 2020.Sep.30  14-55-15 to 2020.Nov.18  19-05-15 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 25):
        file_skipped = pd.read_excel("{}/loco/25/25 from 2020.Feb.05  14-43-59 to 2020.Apr.18  22-03-59 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/25/25 from 2020.Apr.21  14-03-37 to 2020.Aug.10  02-23-37 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/25/25 from 2020.Aug.11  17-01-04 to 2020.Nov.18  19-11-04 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif( animal == 26):
        file_skipped = pd.read_excel("{}/loco/26/26 from 2020.Feb.05  14-43-47 to 2020.Mai.22  12-13-47 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/26/26 from 2020.Jun.01  11-18-24 to 2020.Sep.23  05-13-24 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/26/26 from 2020.Sep.30  14-55-31 to 2020.Nov.18  19-00-31 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 27):
        file_skipped = pd.read_excel("{}/loco/27/27 from 2020.Feb.05  14-43-49 to 2020.Mai.08  02-33-49 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/27/27 from 2020.Mai.14  11-50-42 to 2020.Sep.05  05-45-42 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/27/27 from 2020.Sep.30  14-55-45 to 2020.Nov.18  19-00-45 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 28):
        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Apr.15  15-54-39 to 2020.Apr.28  12-44-39 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Aug.11  17-01-20 to 2020.Aug.26  00-11-20 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Aug.31  11-35-54 to 2020.Sep.14  10-50-54 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Jul.01  12-06-41 to 2020.Jul.15  19-01-41 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Jul.22  13-10-53 to 2020.Aug.03  09-50-53 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Jun.01  11-18-40 to 2020.Jun.17  03-43-40 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
    
        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Mrz.05  15-00-42 to 2020.Mrz.19  09-20-42 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity


        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Mrz.25  13-21-23 to 2020.Apr.10  16-46-23 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Okt.22  14-15-16 to 2020.Nov.05  06-45-16 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

        file_skipped = pd.read_excel("{}/loco/28/28 from 2020.Sep.23  11-56-54 to 2020.Okt.07  15-16-54 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 29):
        file_skipped = pd.read_excel("{}/loco/29/29 from 2020.Feb.10  12-06-08 to 2020.Mai.31  09-26-08 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/29/29 from 2020.Jun.04  14-24-31 to 2020.Sep.25  20-19-31 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/29/29 from 2020.Sep.30  14-55-58 to 2020.Nov.18  19-00-58 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 30):
        file_skipped = pd.read_excel("{}/loco/30/30 from 2020.Feb.10  12-06-17 to 2020.Mai.29  22-56-17 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/30/30 from 2020.Jun.04  14-24-45 to 2020.Aug.11  16-54-45 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 31):
        file_skipped = pd.read_excel("{}/loco/31/31 from 2020.Feb.10  12-06-41 to 2020.Mai.26  00-31-41 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/31/31 from 2020.Jun.04  14-25-02 to 2020.Sep.26  08-20-02 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/31/31 from 2020.Sep.30  14-56-09 to 2020.Nov.18  19-01-09 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    elif ( animal == 32 ):
        file_skipped = pd.read_excel("{}/loco/32/32 from 2020.Feb.17  13-24-14 to 2020.Mai.22  02-44-14 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('{}-%m-%d'.format(str(2020)))
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    
        file_skipped = pd.read_excel("{}/loco/32/32 from 2020.Jun.04  14-25-16 to 2020.Sep.15  21-00-16 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity
    
        file_skipped = pd.read_excel("{}/loco/32/32 from 2020.Sep.22  10-16-38 to 2020.Nov.18  19-01-38 -- 1.xlsx".format(path), skiprows=18)
        reform_length = len(file_skipped)
        for index_ref in range(reform_length):
            lf_date = file_skipped['Date, Time'][index_ref].strftime('%Y-%m-%d')
            lf_time = file_skipped['Date, Time'][index_ref].strftime('%H:%M:{}'.format(str('00')))
            lf_activity = file_skipped['Activity (detection(s))'][index_ref]
            index_value = dataframe[(dataframe['date'] == lf_date ) & (dataframe['time'] == lf_time)].index
            dataframe.loc[index_value, 'locomotive'] = lf_activity

    return(dataframe)

In [ ]:
def writer_(main_df, list_of_them, number, method, a_n, path):
    print(number)
    file = pd.read_excel("{}\{}".format(path, list_of_them[number]), skiprows=35)
    if method == str('dep_onlywater'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i6 in range(len(extracted_a1)):
            ind0 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i6]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i6]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind0, 'water'] = extracted_a1['water'][list_index[i6]]
    elif method == str('ade_only'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i7 in range(len(extracted_a1)):
            ind1 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i7]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i7]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind1, 'water'] = extracted_a1['water'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i7]]
            main_df.loc[ind1, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i7]]
    elif method == str('ade_oxy'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i8 in range(len(extracted_a1)):
            ind2 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i8]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i8]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind2, 'water'] = extracted_a1['water'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i8]]
            main_df.loc[ind2, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i8]]
            main_df.loc[ind2, 'oxytocin'] = str('applied')
    elif method == str('dep_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i9 in range(len(extracted_a1)):
            ind3 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i9]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i9]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind3, 'water'] = extracted_a1['water'][list_index[i9]]
            main_df.loc[ind3, 'quinine'] = str('applied')
    elif method == str('ade_quinine'):
        file_ = file.rename(columns={'Unnamed: 0': 'date', 'Unnamed: 1': 'time', 'Unnamed: 2': 'animal',
                                     'Unnamed: 3': 'box', '[ml]': 'water', '[ml].1': 'alcohol_5%',
                                     '[ml].2': 'alcohol_10%', '[ml].3': 'alcohol_20%'})
        extracted_a1 = file_[file_['box'] == a_n]
        list_index = list(file_[file_['box'] == a_n].index)
        for i10 in range(len(extracted_a1)):
            ind4 = main_df.loc[(main_df['date'] == extracted_a1['date'][list_index[i10]].strftime('%Y-%m-%d')) &
                               (main_df['time'] == extracted_a1['time'][list_index[i10]].strftime('%H:%M:%S'))].index[0]
            main_df.loc[ind4, 'water'] = extracted_a1['water'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_5%'] = extracted_a1['alcohol_5%'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_10%'] = extracted_a1['alcohol_10%'][list_index[i10]]
            main_df.loc[ind4, 'alcohol_20%'] = extracted_a1['alcohol_20%'][list_index[i10]]
            main_df.loc[ind4, 'quinine'] = str('applied')
    else:
        print("{} is non of them".format(list_of_them[number]))
    return main_df

In [ ]:
print("Please enter the animal code:")
animal = int(input())
print("Please enter the box number:")
box = int(input())
print("Please enter the strain of the animal")
strain = input()
print("Please enter the gender of the animal")
gender = input()
print("Please enter the start date:(year)")

In [ ]:
d1 = input()
print("Please enter the start date:(month)")
d2 = input()
print("Please enter the start date:(day)")
d3 = input()
date_i = date(int(d1), int(d2), int(d3))
print("Please enter the end date:(year)")
d4 = input()
print("Please enter the end date:(month)")
d5 = input()
print("Please enter the end date:(day)")
d6 = input()
date_f = date(int(d4), int(d5), int(d6))

In [ ]:
date_i = date(2020, 2, 12)
date_f = date(2020, 11, 17)

delta = date_f - date_i
number_of_days = int(delta.days)+1
print("Experiement was {} days long".format(number_of_days))

total_length = number_of_days * 1440      #length of dataframe for minute time_scale
df = pd.DataFrame(index = range(total_length), columns = ['date', 'time', 'day_index', 'hour_index',
                                                         'animal', 'box', 'strain',
                                                         'state', 'oxytocin', 'quinine', 'water',
                                                         'alcohol_5%', 'alcohol_10%', 'alcohol_20%',
                                                         'locomotive'])


df = state_(df, number_of_days)
df = day_and_hour_index(df, number_of_days)
df['animal'] = animal
df['box'] = box
df['strain'] = strain
df['date'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%Y-%m-%d')
df['time'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%H:%M:%S')

path_ = os.getcwd()
path_ = path_+'\\data\\2020'
onlyfiles = [f for f in listdir("{}".format(path_))
             if isfile(join("{}".format(path_), f))]

In [ ]:
#scrolling over all files for the current animal

df = writer_(df, onlyfiles, 0, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 1, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 2, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 3, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 4, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 5, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 6, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 7, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 8, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 9, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 10, str('ade_only'), box, path_)

df = writer_(df, onlyfiles, 11, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 12, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 13, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 14, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 15, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 16, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 17, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 18, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 19, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 20, str('ade_only'), box, path_)

df = writer_(df, onlyfiles, 21, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 22, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 23, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 24, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 25, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 26, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 27, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 28, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 29, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 30, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 31, str('ade_quinine'), box, path_)
df = writer_(df, onlyfiles, 32, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 33, str('dep_onlywater'), box, path_)

df = writer_(df, onlyfiles, 34, str('ade_quinine'), box, path_)
df = writer_(df, onlyfiles, 35, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 36, str('ade_only'), box, path_)
df = writer_(df, onlyfiles, 37, str('dep_onlywater'), box, path_)
df = writer_(df, onlyfiles, 38, str('ade_oxy'), box, path_)

In [ ]:
df1 = pd.DataFrame(index=range(total_length), columns=['date', 'time', 'day_index', 'hour_index', 'animal', 'box',
                                                       'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                       'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])

In [ ]:
df1 = state_(df1, number_of_days)
df1 = day_and_hour_index(df1, number_of_days)
df1['animal'] = animal
df1['box'] = box
df1['strain'] = strain
df1['date'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%Y-%m-%d')
df1['time'] = pd.date_range("2020-02-12", periods=total_length, freq="T").strftime('%H:%M:%S')
df1['quinine'] = df['quinine']
df1['oxytocin'] = df['oxytocin']
df1['state'] = df['state']

for hour in range(1, number_of_days * 24 + 1):
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['water'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'water'] - df.loc[list_[i], 'water']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'water'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_5%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_5%'] - df.loc[list_[i], 'alcohol_5%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_5%'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_10%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_10%'] - df.loc[list_[i], 'alcohol_10%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_10%'] = temp_val
    df_extracted = df.loc[(df['hour_index'] == hour) & (df['alcohol_20%'].notnull())]
    list_ = np.array(df_extracted.water.index)
    if len(list_) > 1:
        for i in range(len(list_) - 1):
            temp_val = df.loc[list_[i + 1], 'alcohol_20%'] - df.loc[list_[i], 'alcohol_20%']
            if temp_val != 0.0:
                df1.loc[list_[i + 1], 'alcohol_20%'] = temp_val

df1_wl = loco_writer(1, df1, path_)


In [ ]:
threshold_low = 0.08
threshold_up = threshup_()
total_length_h = number_of_days * 24

df_h = pd.DataFrame(index=range(number_of_days * 24), columns=['date', 'time', 'day_index', 'hour_index', 'animal',
                                                               'box', 'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])
df_h = m_day_and_hour_index(df_h, number_of_days)
df_h = state_hour(df_h, number_of_days)

df_h['animal'] = animal
df_h['box'] = box
df_h['strain'] = strain
df_h['date'] = pd.date_range("2020-02-12", periods=number_of_days * 24, freq="H").strftime('%Y-%m-%d')
df_h['time'] = pd.date_range("2020-02-12", periods=number_of_days * 24, freq="H").strftime('%H:%M:%S')

hours = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'quinine'] = str('applied')

hours = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].hour_index)))
for i in range(len(hours)):
    df_h.loc[hours[i]-1, 'oxytocin'] = str('applied')


for hour in range(1, number_of_days*24+1):
    w_w = wistar_w(gender, hour)
    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            df_h.loc[ind, 'water'] = _temp
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up ):
            temp__ = real_consump(_temp, w_w, 5)
            df_h.loc[ind, 'alcohol_5%'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 10)
            df_h.loc[ind, 'alcohol_10%'] = temp__
            _temp = np.nan

    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1.loc[(df1['hour_index'] == hour) & (df1['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr[temp_arr > threshold_low]) > 1:
        _temp = np.nansum(temp_arr[temp_arr > threshold_low])
        if (_temp < threshold_up):
            temp__ = real_consump(_temp, w_w, 20)
            df_h.loc[ind, 'alcohol_20%'] = temp__
            _temp = np.nan

number_of_days = 280
for hour in range(1, number_of_days*24+1):
    list_ = []
    ind = np.array(df_h.loc[(df_h['hour_index'] == hour)].index)
    df_extracted = df1_wl.loc[(df1_wl['hour_index'] == hour) & (df1_wl['locomotive'].notnull())]
    temp_arr = np.array(df_extracted['locomotive'])
    if len(temp_arr) > 1:
        df_h.loc[ind, 'locomotive'] = np.nansum(temp_arr)

In [ ]:
df_d_light = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                                'strain', 'state', 'oxytocin', 'quinine', 'water',
                                                                'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])

df_d_dark = pd.DataFrame(index=range(number_of_days), columns=['date', 'day_index', 'animal', 'box',
                                                               'strain', 'state', 'oxytocin', 'quinine',
                                                               'water', 'alcohol_5%', 'alcohol_10%', 'alcohol_20%', 'locomotive'])

df_d_light = d_day_index(df_d_light, number_of_days)
df_d_dark = d_day_index(df_d_dark, number_of_days)

df_d_light['animal'] = animal
df_d_light['box'] = box
df_d_light['strain'] = strain
df_d_light['state'] = str('light')
df_d_light['date'] = pd.date_range("2020-02-12", periods=number_of_days, freq="D").strftime('%Y-%m-%d')

df_d_dark['animal'] = animal
df_d_dark['box'] = box
df_d_dark['strain'] = strain
df_d_dark['state'] = str('dark')
df_d_dark['date'] = pd.date_range("2020-02-12", periods=number_of_days, freq="D").strftime('%Y-%m-%d')


days = sorted(set(np.array(df1.loc[(df1['quinine'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'quinine'] = str('applied')
    df_d_light.loc[days[i]-1, 'quinine'] = str('applied')

days = sorted(set(np.array(df1.loc[(df1['oxytocin'] == str('applied'))].day_index)))
for i in range(len(days)):
    df_d_dark.loc[days[i]-1, 'oxytocin'] = str('applied')
    df_d_light.loc[days[i]-1, 'oxytocin'] = str('applied')


for day in range(1, number_of_days+1):
    list_ = []
    ind = np.array(df_d_light.loc[(df_d_light['day_index'] == day)].index)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'water'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['water'].notnull())]
    temp_arr = np.array(df_extracted['water'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'water'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_5%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_5%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_5%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_5%'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_10%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_10%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_10%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_10%'] = np.nansum(temp_arr)

    list_ = []

    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('light')) & (df_h['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'alcohol_20%'] = np.nansum(temp_arr)
    df_extracted = df_h.loc[(df_h['day_index'] == day) & (df_h['state'] == str('dark')) & (df_h['alcohol_20%'].notnull())]
    temp_arr = np.array(df_extracted['alcohol_20%'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'alcohol_20%'] = np.nansum(temp_arr)

for day in range(1, number_of_days+1):
    list_ = []
    ind = np.array(df_d_light.loc[(df_d_light['day_index'] == day)].index)
    df_extracted = df1_wl.loc[(df1_wl['day_index'] == day) & (df1_wl['state'] == str('light')) & (df1_wl['locomotive'].notnull())]
    temp_arr = np.array(df_extracted['locomotive'])
    if len(temp_arr) > 1:
        df_d_light.loc[ind, 'locomotive'] = np.nansum(temp_arr)
    df_extracted = df1_wl.loc[(df1_wl['day_index'] == day) & (df1_wl['state'] == str('dark')) & (df1_wl['locomotive'].notnull())]
    temp_arr = np.array(df_extracted['locomotive'])
    if len(temp_arr) > 1:
        df_d_dark.loc[ind, 'locomotive'] = np.nansum(temp_arr)

In [ ]:
df.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_raw.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df1_wl.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_consumption.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df1.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_consumptiontt.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_h.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_{}_hourly.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_dark.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_dark.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)
df_d_light.to_excel(r'{}\dataframes\drinkometer_df_{}_{}_{}_daily_light.xlsx'.format(path_, strain, gender, animal, box), index=False, header=True)